# COMP47590: Advanced Machine Learning
# Assignment 1: Multi-label Classification

Name(s):<br>
<b>Kunal J. Tolani</b> &
<b>Chirag K. Shah</b><br>
Student Number(s):<br>
<b>19200153</b> &
<b>19200072</b>

## Importing required packages

In [1]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn import metrics
from sklearn.metrics import accuracy_score, make_scorer, f1_score


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.base import clone
from random import shuffle

## Task 0: Load the Yeast Dataset

In [2]:
yeast = pd.read_csv('yeast.csv')
yeast

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,0,0,1,1,0,0,0,1,1,0
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,0,0,0,0,0,0,0,0,0,0
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,0,0,0,0,0,0,0,1,1,0
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,0,0,0,0,0,0,0,0,0,0
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412,-0.119784,0.001259,-0.123645,-0.015513,-0.059683,0.091032,-0.043302,0.229219,-0.071498,0.182709,...,0,0,0,0,0,0,0,0,0,0
2413,0.085327,0.058590,0.085268,-0.020897,0.068972,0.030125,0.078056,0.011346,0.052618,0.066093,...,0,0,0,0,0,0,0,1,1,0
2414,0.082526,-0.095571,-0.022019,-0.046793,-0.038360,0.041084,0.056509,0.011749,-0.029657,-0.012198,...,0,1,1,1,0,0,0,1,1,0
2415,-0.130830,0.008868,-0.009457,-0.058930,-0.041224,0.042269,0.117717,0.037388,-0.085563,0.136649,...,0,0,0,0,0,0,0,1,1,0


In [3]:
X = yeast.iloc[:, 0:-14]

In [4]:
y = yeast.iloc[:,-14:]
y

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0,0,0,0,0,0,1,1,0,0,0,1,1,0
1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,1,1,0
3,0,0,1,1,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412,0,1,1,0,0,0,0,0,0,0,0,0,0,0
2413,1,1,0,0,0,0,0,0,0,0,0,1,1,0
2414,0,0,0,0,0,1,1,1,0,0,0,1,1,0
2415,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [5]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.7)

In [6]:
# Custom accuracy score based on hamming loss implementation

def get_accuracy_score(y_test,y_pred):
    
    if 'numpy' not in str(type(y_pred)):
        y_pred = y_pred.to_numpy()
        
    if 'numpy' not in str(type(y_test)):
        y_test = y_test.to_numpy()
    
    
    assert(y_test.shape == y_pred.shape)
    
    if y_pred.shape[1] <= 5: #For a smaller number of labels, a ratio of half the labels being correct is good enough
        ratio = 0.5
    else:
        ratio = 0.7 #For a number of labels, at least 70% of the predicted labels must be correct
    
    acc_rows = []
    
    for i in range(len(y_test)):
        acc_rows.append(np.count_nonzero(y_test[i]==y_pred[i]))
#         acc_rows.append(np.count_nonzero(y_test.iloc[i,:].values==y_pred.iloc[i,:].values)) #Count the number of matches
        
    acc_rows = [1 if x/y_pred.shape[1] >= ratio else 0 for x in acc_rows] #1 if ratio of match in a row is greater than ratio, else 0
    return sum(acc_rows)/len(acc_rows) # Mean accuracy


## Task 1: Implement the Binary Relevance Algorithm

In [7]:
class BinaryRelevanceClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, base_model=LogisticRegression()):
        self.base_model = base_model #base model - by default logistic regression
    
    def fit(self, X, y):
        self.model_list_ = []
        for column in y:  #For every label in y, fit the model independently
            clf = clone(self.base_model) #Clone the base model to avoid shallow copy
            A,b = check_X_y(X,y[column])
            clf.fit(A,b) #fit the model for the particular label value
            self.model_list_.append(clf) #Add the model to the saved model list
            
    def predict(self,X):
        check_is_fitted(self, ['model_list_']) 
        X = check_array(X)
        y_pred = pd.DataFrame() #Create a dataframe to save predictions
        i = 1
        for model in self.model_list_: #Make predictions for each label, using the corresponding mdodel
            y_pred_class = model.predict(X) 
            y_pred[i] = y_pred_class #Add current prediction to dataframe
            i+=1
        return y_pred.to_numpy() #return final predictions
    
    def predict_proba(self,X):
        check_is_fitted(self, ['model_list_']) #Check if model list is present
        X = check_array(X)
        y_pred = pd.DataFrame()
        i = 1
        for model in self.model_list_:
            y_pred_class = model.predict_proba(X) #Call predict_proba of the each base model
            y_pred[i] = [one_prob[1] for one_prob in y_pred_class] #Add the probability of 1 to the dataframe
            i+=1
        return y_pred.to_numpy() #return final probabilities

In [8]:
binclf = BinaryRelevanceClassifier(base_model=RandomForestClassifier())
binclf.fit(X_train,y_train)
y_pred = binclf.predict(X_test) # Predictions on test data

In [9]:
y_pred.shape

(726, 14)

In [10]:
display(get_accuracy_score(y_test,y_pred)) # Default accuracy score not used because it performs exact matching
display(f1_score(y_test, y_pred, average='macro')) # F1 Score

0.8140495867768595

0.3539791385313155

In [11]:
# Grid Search with multiple base estimators
params_grid = [
    {
                'base_model':[DecisionTreeClassifier()],
                'base_model__criterion': ["gini", "entropy"],
                'base_model__max_depth': [None, 10, 15, 20, 50],
                'base_model__min_samples_split':[2,8,10]
                },
                {
                'base_model': [RandomForestClassifier()],
                'base_model__min_samples_split':[2,8,10],
                'base_model__n_estimators':[100,500],
                },
                {
                'base_model':[LogisticRegression()],
                'base_model__max_iter':[1000,20000,40000]
                }
              ]

# Perform the search
my_tuned_model = GridSearchCV(BinaryRelevanceClassifier(), params_grid, verbose = 2, n_jobs=-1, scoring=make_scorer(get_accuracy_score))
my_tuned_model.fit(X_train, y_train)


Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:  6.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=BinaryRelevanceClassifier(base_model=LogisticRegression(C=1.0,
                                                                               class_weight=None,
                                                                               dual=False,
                                                                               fit_intercept=True,
                                                                               intercept_scaling=1,
                                                                               l1_ratio=None,
                                                                               max_iter=100,
                                                                               multi_class='auto',
                                                                               n_jobs=None,
                                                                               penalty='l2',
                

In [12]:
#Finding our best params and score
print(my_tuned_model.best_params_)
print(my_tuned_model.best_score_)

{'base_model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'base_model__min_samples_split': 8, 'base_model__n_estimators': 500}
0.835010734670367


## Task 2: Implement the Binary Relevance Algorithm with Under-Sampling

In [13]:
# Write your code here
class BinaryRelevanceClassifierUnderSampled(BaseEstimator, ClassifierMixin):
    
    def __init__(self, base_model=LogisticRegression()):
        self.base_model = base_model
    
    def fit(self, X, y):
        
        self.model_list_ = []
        for column in y: #For every label in y, fit the model independently
            X_copy = X.copy()
            clf = clone(self.base_model) #Clone the base model to avoid shallow copy
            concat_x_y = X_copy.join(y[column]) #Concat X and the label to perform undersampling
            label_0, label_1 = sum(y[column]==0), sum(y[column])
            
            ratio = label_1/label_0 #Check the ratio of 1s to 0s in a target
            
            '''Here, we undersample only when the difference between 1 and 0 is quite large, 
            that is if the number of the majority class is more than double the number of the minority class'''
            
            if ratio < 0.5: 
                xy_label_1 = concat_x_y[concat_x_y[column]==1]
                xy_label_0 = concat_x_y[concat_x_y[column]==0].sample(label_1) # Undersample using pd.sample on label 0
                concat_x_y = pd.concat([xy_label_0, xy_label_1], axis=0)
                
            elif ratio >= 2:
                
                xy_label_1 = concat_x_y[concat_x_y[column]==1].sample(label_0) # Undersample using pd.sample on label 1
                xy_label_0 = concat_x_y[concat_x_y[column]==0]
                concat_x_y = pd.concat([xy_label_0, xy_label_1], axis=0)              
            
            #Separate concatenated dataset into X and y
            X_copy = concat_x_y.iloc[:,:-1]
            y_column = concat_x_y.iloc[:,-1]
            
            #Fit the model
            A,b = check_X_y(X_copy,y_column)
            clf.fit(A,b)
            self.model_list_.append(clf) # Append model to model list
            
    def predict(self,X):
        check_is_fitted(self, ['model_list_']) # Check if the model list is present
        X = check_array(X)
        y_pred = pd.DataFrame() # Create a dataframe to save predictions
        i = 1
        for model in self.model_list_: # Make predictions for each label, using the corresponding mdodel
            y_pred_class = model.predict(X) 
            y_pred[i] = y_pred_class # Add current prediction to dataframe
            i+=1
        return y_pred.to_numpy() # Return predictions
    
    def predict_proba(self,X):
        check_is_fitted(self, ['model_list_']) #Check if the model list is present
        X = check_array(X)
        y_pred = pd.DataFrame()
        i = 1
        for model in self.model_list_:
            y_pred_class = model.predict_proba(X) #Call predict_proba of the each base model
            y_pred[i] = [one_prob[1] for one_prob in y_pred_class] #Add the probabilities of 1 to the dataframe
            i+=1
        return y_pred.to_numpy() #return probabilities

In [14]:
binclf_u = BinaryRelevanceClassifierUnderSampled(base_model=RandomForestClassifier())
binclf_u.fit(X_train,y_train)
y_pred_un = binclf_u.predict(X_test) # Predictions on test data

In [15]:
y_pred_un.shape

(726, 14)

In [16]:
display(get_accuracy_score(y_test,y_pred_un))
display(f1_score(y_pred_un,y_test,average='macro'))

0.4628099173553719

0.44454831468970496

In [17]:
# Grid Search with multiple base estimators
params_grid = [
    {
                'base_model':[DecisionTreeClassifier()],
                'base_model__criterion': ["gini", "entropy"],
                'base_model__max_depth': [None, 10, 15, 20, 50],
                'base_model__min_samples_split':[2,8,10]
                },
                {
                'base_model': [RandomForestClassifier()],
                'base_model__min_samples_split':[2,8,10],
                'base_model__n_estimators':[100,500],
                },
                {
                'base_model':[LogisticRegression()],
                'base_model__max_iter':[1000,20000,40000]
                }
              ]

# Perform the search
my_tuned_model = GridSearchCV(BinaryRelevanceClassifierUnderSampled(), params_grid, verbose = 2, n_jobs=-1, scoring=make_scorer(get_accuracy_score))
my_tuned_model.fit(X_train, y_train)


Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:  2.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=BinaryRelevanceClassifierUnderSampled(base_model=LogisticRegression(C=1.0,
                                                                                           class_weight=None,
                                                                                           dual=False,
                                                                                           fit_intercept=True,
                                                                                           intercept_scaling=1,
                                                                                           l1_ratio=None,
                                                                                           max_iter=100,
                                                                                           multi_class='auto',
                                                                                           n_jobs=None,
 

In [18]:
#Finding our best params and score
print(my_tuned_model.best_params_)
print(my_tuned_model.best_score_)

{'base_model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'base_model__min_samples_split': 2, 'base_model__n_estimators': 100}
0.44294042694314995


## Task 3: Compare the Performance of Different Binary Relevance Approaches

In [19]:
# Using the best model of binary relevance undersampled for evaluation across models

classifier = my_tuned_model.best_estimator_.base_model
    
print(classifier)
print("Without Under Sampling")
binclf = BinaryRelevanceClassifier(base_model=classifier)
binclf.fit(X_train,y_train)
y_pred = binclf.predict(X_test)

print("\nAccuracy: " + str(get_accuracy_score(y_test,y_pred)))
print("F1 Score: " + str(f1_score(y_test, y_pred, average='macro')))
print("\n") 
print("With Under Sampling")

binclf_u = BinaryRelevanceClassifierUnderSampled(base_model=classifier)
binclf_u.fit(X_train,y_train)
y_pred_un = binclf_u.predict(X_test)
print("Accuracy: " + str(get_accuracy_score(y_test,y_pred_un)))
print("F1 Score: " + str(f1_score(y_test, y_pred_un, average='macro')))
print("\n") 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Without Under Sampling

Accuracy: 0.8168044077134986
F1 Score: 0.3476054358289854


With Under Sampling
Accuracy: 0.44352617079889806
F1 Score: 0.44166645546409666




## Task 4: Implement the Classifier Chains Algorithm

In [20]:
class ClassifierChains(BaseEstimator, ClassifierMixin):
    
    def __init__(self, base_model=LogisticRegression(), order = None, undersample=False):
        self.base_model = base_model # The base estimator
        self.order = order # Order of labels in which the labels are to be sent to the classifier
        self.undersample = undersample # Whether or not to undersample
        
    def fit(self, X, y):
        X_cpy = X.copy()
        y_cpy = y.copy()
        self.order_shuffle = None
        self.base_order = list(range(len(y.columns)))
        
        if self.order is None:
            self.order_shuffle = list(range(len(y_cpy.columns))) # Same order as input
            
        elif self.order == 'random':
            self.order_shuffle = list(range(len(y_cpy.columns))) # Random label order
            random.shuffle(self.order_shuffle)
            print(self.order_shuffle)
            
        else:
            if len(self.order) == len(y.columns) and all(isinstance(item, int) for item in self.order):
                self.order_shuffle = self.order # Label order given by user
        
        y_cpy = y_cpy.iloc[:,self.order_shuffle] # Shuffle y according to label order
        
        
        self.model_list_ = []
        for column in y_cpy:
            X_cpy.reset_index(drop=True, inplace=True)
            y_cpy.reset_index(drop=True, inplace=True)
            
            X_copy = X_cpy.copy()
            
            if self.undersample:
                concat_x_y = X_copy.join(y_cpy[column]) #Concat X and the label to perform undersampling
                label_0, label_1 = sum(y_cpy[column]==0), sum(y_cpy[column])

                ratio = label_1/label_0 #Check the ratio of 1s to 0s in a target

                '''Here, we undersample only when the difference between 1 and 0 is quite large, 
                that is if the number of the majority class is more than double the number of the minority class'''

                if ratio < 0.5: # Undersample using pd.sample on label 0
                    xy_label_1 = concat_x_y[concat_x_y[column]==1]
                    xy_label_0 = concat_x_y[concat_x_y[column]==0].sample(label_1)
                    concat_x_y = pd.concat([xy_label_0, xy_label_1], axis=0)

                elif ratio >= 2: # Undersample using pd.sample on label 1
                    xy_label_1 = concat_x_y[concat_x_y[column]==1].sample(label_0) 
                    xy_label_0 = concat_x_y[concat_x_y[column]==0]
                    concat_x_y = pd.concat([xy_label_0, xy_label_1], axis=0)              

                #Separate concatenated dataset into X and y
                X_copy = concat_x_y.iloc[:,:-1]
                y_column = concat_x_y.iloc[:,-1]

            else:
                y_column = y_cpy[column]
                
            clf = clone(self.base_model)
            A,b = check_X_y(X_copy, y_column)
            clf.fit(A,b)  # Fit model to the (undersampled if true) dataset
            self.model_list_.append(clf)   # Append model to list
            y_col = pd.DataFrame(y_cpy[column])
            X_cpy = pd.concat([X_cpy, y_col], axis=1)  # Concatenate current label to X
            
            
    def predict(self,X):
        X_cpy = X.copy()
        X_cpy.reset_index(drop=True, inplace=True)
        check_is_fitted(self, ['model_list_']) # Check if the model list is present
        y_pred = pd.DataFrame() # Create a dataframe to save predictions
        i = 0
        for model in self.model_list_: # Make predictions for each label, using the corresponding mdodel
            X_cpy = check_array(X_cpy)
            y_pred_class = model.predict(X_cpy) # Predict current label
            y_pred[self.order_shuffle[i]] = y_pred_class # Save the predicted class according to our label order
            
            X_cpy = np.column_stack((X_cpy, y_pred_class)) # Append the predicted column to X test
            i+=1
            
        y_pred = y_pred.loc[:,self.base_order] # Return according to original order
        return y_pred.to_numpy() # Return as numpy array
    
    def predict_proba(self,X):
        X_cpy = X.copy()
        X_cpy.reset_index(drop=True, inplace=True)
        check_is_fitted(self, ['model_list_']) #Check if the model list is present
        y_pred = pd.DataFrame() # Create a dataframe to save predictions
        i = 0
        for model in self.model_list_: # Make prediction probabilities for each label, using the corresponding mdodel
            X_cpy = check_array(X_cpy)
            y_pred_class = model.predict(X_cpy) # Predict current label
            y_pred_class_proba = model.predict_proba(X_cpy) # Predict probabilties using current model
            y_pred[self.order_shuffle[i]] = [one_prob[1] for one_prob in y_pred_class_proba] # Save the probabilities of 1 according to our label order
            X_cpy = np.column_stack((X_cpy, y_pred_class)) # Append prediction to X test
            i+=1
        y_pred = y_pred.loc[:,self.base_order] # Return probabilities according to original order
        return y_pred.to_numpy() # Return as numpy

In [21]:
class_chain = ClassifierChains(base_model=RandomForestClassifier(),undersample=False)
class_chain.fit(X_train,y_train)
y_pred_cc = class_chain.predict(X_test) # Predictions on test data

In [22]:
display(get_accuracy_score(y_test,y_pred_cc))
display(f1_score(y_pred_cc,y_test,average='macro'))

0.8471074380165289

0.3763361468097855

In [23]:
y_pred_cc.shape

(726, 14)

In [24]:
#Performing grid search with different base estimators
params_grid = [
    {
                'base_model':[DecisionTreeClassifier()],
                'base_model__criterion': ["gini", "entropy"],
                'base_model__max_depth': [None, 10, 15, 20, 50],
                'base_model__min_samples_split':[2,8,10]
                },
                {
                'base_model': [RandomForestClassifier()],
                'base_model__min_samples_split':[2,10],
                'base_model__n_estimators':[100,500],
                'order':['random','random','random','random']
                },
                {
                'base_model':[LogisticRegression()],
                'base_model__max_iter':[1000,20000,40000]
                }
              ]

# Perform the search
my_tuned_model = GridSearchCV(ClassifierChains(), params_grid, verbose = 10, n_jobs=-1, scoring=make_scorer(get_accuracy_score))
my_tuned_model.fit(X_train, y_train)


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   34.6s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5

[6, 10, 1, 8, 5, 4, 11, 2, 13, 3, 0, 12, 9, 7]


GridSearchCV(cv=None, error_score=nan,
             estimator=ClassifierChains(base_model=LogisticRegression(C=1.0,
                                                                      class_weight=None,
                                                                      dual=False,
                                                                      fit_intercept=True,
                                                                      intercept_scaling=1,
                                                                      l1_ratio=None,
                                                                      max_iter=100,
                                                                      multi_class='auto',
                                                                      n_jobs=None,
                                                                      penalty='l2',
                                                                      random_state=None,
                 

In [25]:
#Finding our best params and score
print(my_tuned_model.best_params_)
print(my_tuned_model.best_score_)

{'base_model': RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False), 'base_model__min_samples_split': 2, 'base_model__n_estimators': 500, 'order': 'random'}
0.842106089961774


### Implementing Monte Carlo Methods for finding best label orders for Classifier Chains
Based on (Read, Martino and Luengo, 2014)

In [26]:
class MonteClassifierChains(BaseEstimator, ClassifierMixin):
    
    def __init__(self, base_model=LogisticRegression()):
        self.base_model = base_model
        self.labels_id_dict = {}
        self.best_label_order = []
        self.original_label_order = []
    
    def fit(self, X, y):
        
        X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, train_size = 0.7)
        
        labels = list(y.columns)
        self.original_label_order = list(y.columns)
        replace_set = set()
        list_labels = []
        payoff_list = []
    
        #Swapper function for swapping two randomly selected labels
        def swapper(labels, weights=None):
            a, b = np.random.choice(labels, size=2, p=weights, replace=False) # Swap two labels according to the probabilities
            m = labels.index(a)
            n = labels.index(b)
            labels[m] = b
            labels[n] = a
            return labels, a, b #Return the label list, and the swapped labels
        
        # Payoff returns the accuracy on a validation set
        def payoff(list_order, X_val, y_test):
        
            X_val.reset_index(drop=True, inplace=True) # Reset index - for concatenation later
            y_pred_val_df = pd.DataFrame()
            
            for i in range(len(list_order)):

                try:
                    clf = [key for key, value in self.labels_id_dict.items() if value == list_order[:i+1]][0]            
                except IndexError:
                    return float('inf') #Return infinity is label order not found

                y_pred_val = clf.predict(X_val)
                X_val = np.column_stack((X_val, y_pred_val))
                y_pred_val_df[list_order[i]] = y_pred_val

            return get_accuracy_score(y_test, y_pred_val_df)
        

        count = 0 #Count iterations
        weights = [1/len(labels) for i in labels] #Probability weights for labels
        
        while count < 100:
        

            if count >= 20:  #After 20 iterations, keep updating probabilities according to position in index
                beta = 0.5
                t = count
                weights = [(1/len(labels))**(beta*t/(labels.index(i)+1)) for i in labels]
                weights = [float(i)/sum(weights) for i in weights]    
            
            X_cpy = X_train.copy()
            y_cpy = y_train.copy()
            X_cpy.reset_index(drop=True, inplace=True)
        
            label_list_for = []
            l,p,q = swapper(labels, weights)
            if p in replace_set and q in replace_set:   #Swap only if both p and q have not already been swapped
                    continue
            
            y_cpy = y_cpy.loc[:,l]
            index_p = l.index(p)
            index_q = l.index(q)
            y_cpy.reset_index(drop=True, inplace=True)
                    
            
            if count > 0:
                smaller_index = index_p if index_p < index_q else index_q
                X_cpy = pd.concat([X_cpy, y_cpy.iloc[:,:smaller_index]], axis=1)
                y_cpy = y_cpy.iloc[:,smaller_index:]   
                label_list_for.extend(l[:smaller_index])
                
                
            for column in y_cpy:  #Classifier Chains
                
                label_list_for.append(column)
                A,b = check_X_y(X_cpy, y_cpy[column])
                clf = clone(self.base_model)
                clf.fit(A,b)
                self.labels_id_dict[clf] = label_list_for.copy()  #Adding list of labels to dict - with classifier id
                b = pd.DataFrame(b)
                                
                X_cpy = pd.concat([X_cpy, b], axis=1)
            
            
            curr_payoff = payoff(label_list_for, X_val, y_val)
            
            if curr_payoff == float('inf'):  #Handling the case when an exception is thrown in payoff
                continue
            if count == 0:
                payoff_list.append(curr_payoff) #Initially not accounting for previous payoff
            else:
                if curr_payoff > payoff_list[-1]:
                    payoff_list.append(curr_payoff)
                else:
                    count+=1
                    continue   #Do not add to final list if payoff is worse

            
            #Add to set of swapped labels, and append a copy of the final list
            replace_set.add(p)
            replace_set.add(q)
            list_labels.append(l.copy())
            count += 1
            labels = l
            
            #Clear replace set every 5 iterations
            if count%5 == 0:
                replace_set.clear()
                
        self.best_label_order = list_labels[-1]    #Final list in the list of labels is the best order
        print(f"Best order found: {self.best_label_order}")
                        
    def predict(self, X):
        
        check_is_fitted(self, ['best_label_order']) # Check if fitted - is best label order generated
        
        X_cpy = X.copy()
        X_cpy.reset_index(drop=True, inplace=True)
        y_pred_val_df = pd.DataFrame()
        
        for i in range(len(self.best_label_order)):
            
            X_cpy = check_array(X_cpy)
            clf = [key for key, value in self.labels_id_dict.items() if value == self.best_label_order[:i+1]][0] # Find the model in dictionary according to current label
            y_pred_val = clf.predict(X_cpy) # Make prediction
            X_cpy = np.column_stack((X_cpy, y_pred_val)) # Add predicted value to X test
            y_pred_val_df[self.best_label_order[i]] = y_pred_val # Add predicted value according to best label order
        
        y_pred_val_df = y_pred_val_df.loc[:,self.original_label_order] # Return predictions based on original order
        
        
        return y_pred_val_df.to_numpy()
    
    def predict_proba(self, X):
        
        check_is_fitted(self, ['best_label_order']) # Check if fitted - is best label order generated
        
        X_cpy = X.copy()
        X_cpy.reset_index(drop=True, inplace=True)
        y_pred_val_df = pd.DataFrame()
        
        for i in range(len(self.best_label_order)):
            X_cpy = check_array(X_cpy)
            clf = [key for key, value in self.labels_id_dict.items() if value == self.best_label_order[:i+1]][0]
            y_pred_val = clf.predict_proba(X_cpy) # Generate probabilities for current label
            X_cpy = np.column_stack((X_cpy, clf.predict(X_cpy))) # Add predicted value to X test
            y_pred_val_df[self.best_label_order[i]] = [one_prob[1] for one_prob in y_pred_val] # Add predicted probabilities of 1 according to best label order

         
        y_pred_val_df = y_pred_val_df.loc[:,self.original_label_order] # Return probabilities based on original order
        
        return y_pred_val_df.to_numpy()
   

In [27]:
class_chain_mcc = MonteClassifierChains(base_model=my_tuned_model.best_estimator_.base_model)
class_chain_mcc.fit(X_train,y_train)
y_pred_mcc = class_chain_mcc.predict(X_test)

Best order found: ['Class11', 'Class2', 'Class3', 'Class4', 'Class5', 'Class6', 'Class7', 'Class8', 'Class9', 'Class10', 'Class1', 'Class12', 'Class13', 'Class14']


In [28]:
display(get_accuracy_score(y_test,y_pred_mcc))
display(f1_score(y_pred_mcc,y_test,average='macro'))

0.8484848484848485

0.36394890032715105

In [29]:
y_pred_mcc.shape

(726, 14)

## Task 5: Evaluate the Performance of the Classifier Chains Algorithm

In [30]:
# Using the best model for classifier chains for evaluation across models

classifier = my_tuned_model.best_estimator_.base_model

print(classifier)
print("BR Without Under Sampling")
binclf = BinaryRelevanceClassifier(base_model=classifier)
binclf.fit(X_train,y_train)
y_pred = binclf.predict(X_test)

print("\nAccuracy: " + str(get_accuracy_score(y_test,y_pred)))
print("F1 Score: " + str(f1_score(y_test, y_pred, average='macro')))
print("\n") 

print("BR With Under Sampling")

binclf_u = BinaryRelevanceClassifierUnderSampled(base_model=classifier)
binclf_u.fit(X_train,y_train)
y_pred_un = binclf_u.predict(X_test)
print("Accuracy: " + str(get_accuracy_score(y_test,y_pred_un)))
print("F1 Score: " + str(f1_score(y_test, y_pred_un, average='macro')))
print("\n") 

print("Classifier Chains Without Under Sampling")

class_cc_model = ClassifierChains(base_model=classifier, undersample=False)
class_cc_model.fit(X_train,y_train)
y_pred_cc = class_cc_model.predict(X_test)
print("Accuracy: " + str(get_accuracy_score(y_test,y_pred_cc)))
print("F1 Score: " + str(f1_score(y_test, y_pred_cc, average='macro')))
print("\n")

print("Classifier Chains With Under Sampling")

class_cc_model_un = ClassifierChains(base_model=classifier, undersample=True)
class_cc_model_un.fit(X_train,y_train)
y_pred_cc_un = class_cc_model_un.predict(X_test)
print("Accuracy: " + str(get_accuracy_score(y_test,y_pred_cc_un)))
print("F1 Score: " + str(f1_score(y_test, y_pred_cc_un, average='macro')))
print("\n")

print("Classifier Chains With Monte Carlo Method")

print("Accuracy: " + str(get_accuracy_score(y_test,y_pred_mcc)))
print("F1 Score: " + str(f1_score(y_test, y_pred_mcc, average='macro')))
print("\n")

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
BR Without Under Sampling

Accuracy: 0.8264462809917356
F1 Score: 0.3513414532898969


BR With Under Sampling
Accuracy: 0.47796143250688705
F1 Score: 0.4496869252917108


Classifier Chains Without Under Sampling
Accuracy: 0.8443526170798898
F1 Score: 0.3805947840081686


Classifier Chains With Under Sampling
Accuracy: 0.4972451790633609
F1 Score: 0.4550165657812229


Classifier Chains With Monte Carlo Method
Accuracy: 0.8484848484848485
F1 Score: 0.

## Task 6: Reflect on the Performance of the Different Models Evaluated

#### Let,  
A: Binary Relevance Classifier  
B: Classifier Chains  
C: Classifier Chains with Monte-Carlo  

#### From the above experiments, we conclude:  
* A without undersampling gives an accuracy of 83.19 which is better than A with undersampling. We undersample the majority class only when the difference between majority and miority is too large. Due to this, valid training samples maybe lost resulting in low accuracy scores.


* A without undersampling has lower F1 scores becuase of high bias which corresponds to high precision and low recall values. However, after undersampling the bias is reduced to an extent which results in a better F1 score.


* B works better than A because it takes label dependency into consideration whereas A doesn't. But B. has more complexity than A. because of the need to use the previous target labels for next predictions. The difference in terms of accuracy is not great and this comes as a surprise. So, for faster computation on Yeast Data, Binary Relevance Classifier can be an adequate method. 

* B with undersampling doesn't outperform than B without undersampling because of the same reasons mentioned in comparison of A.


* Since, Classifier Chains takes previous labels and previous predictions in consideration while performing current predictions, label ordering is important. To find best label orderings, exhaustive search would be too computationally expensive. Hence, we adopted the Monte Carlo method C.


* In this algorithm, we randomly swap two labels at every iteration and calculate payoff. The new label order is only accepted if the payoff at current instance is greater than payoff at previous instance. For a few initial iterations, swaps are performed uniformly accross all labels. However after fixed set of iterations, we assign higher weights to later labels so the entire label order wouldn't be retrained continously as we are freezing models and reusing them at each traversal across iterations. The best order found in the Monte Carlo method is not consistent across runs but is usually just one swap away from the original order.

Performance - **C > B > A**

Complexity - **C > B > A**

<h2>References</h2>

Read, J., Martino, L. and Luengo, D. (2014). Efficient monte carlo methods for multi-dimensional learning with classifier chains. Pattern Recognition, [online] 47(3), pp.1535-1546. Available at: https://arxiv.org/pdf/1211.2190.pdf.  

Read, J., Pfahringer, B., Holmes, G. and Frank, E. (2011). Classifier chains for multi-label classification. Machine Learning, [online] 85(3), pp.333-359. Available at: https://link.springer.com/content/pdf/10.1007/s10994-011-5256-5.pdf.